In [1]:
import torch
import numpy
import matplotlib.pyplot as plt

In [2]:
print(torch.cuda.get_device_name(0))

Tesla V100-SXM2-16GB


In [4]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-gcimid7a
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-gcimid7a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 13.1MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=1b93ae3786ef4374a424ac76dfde5f79ce33493a67950226d1387673b74490a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-e4fzhvvg/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 13.3MB/s 
     |████████████████████████████████| 245kB 18.9MB/s 
     |████████████████████████████████| 112kB 28.3MB/s 


In [5]:
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaModelWithHeads
import numpy as np
import argparse
import pdb

In [7]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/HP/output/step_one/"
}

In [8]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
# loading the data
train_data = datasets.load_dataset("json",data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset("json", data_files=args_dict["val_dataset"])["train"] 
test_data = datasets.load_dataset("json", data_files=args_dict["test_dataset"])["train"] 

#assign an integer key for each label
label_keys = {}
num_labels = 0
for label in train_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

for label in test_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True)
    tokenized_batch["label"] = [label_keys[label] for label in batched_text["label"]]
    return tokenized_batch

train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data))
val_data = val_data.map(tokenization, batched=True, batch_size=len(val_data))
test_data = test_data.map(tokenization, batched=True, batch_size=len(test_data))

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Using custom data configuration default-e6562f94fc99a76a


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e6562f94fc99a76a/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-034d47800bfe2ca3


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-034d47800bfe2ca3/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-9af14c49e244bd15


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9af14c49e244bd15/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [12]:
f1_scores = []
for seed in range(1,4):
  #create model
  model_prev = RobertaForSequenceClassification.from_pretrained(
        args_dict["model"], hidden_dropout_prob=0.1, num_labels=num_labels)
  model_prev_configuration = model_prev.config

  model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=model_prev_configuration,
  )


  adapter_name = "adapter_HP_" + str(seed)
  # this part is for adding the adapter
  model.add_adapter(adapter_name)
  # Add a matching classification head
  model.add_classification_head(
      adapter_name,
      num_labels=num_labels,
    )
  # Activate the adapter
  model.train_adapter(adapter_name)
    

  training_args = TrainingArguments(
      num_train_epochs=15,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      learning_rate=5e-5,
      lr_scheduler_type="constant",
      fp16=True,
      eval_accumulation_steps=20,
      save_strategy="no",
      save_steps=5000,
      save_total_limit=1,
      load_best_model_at_end=False,
      metric_for_best_model="f1",
      evaluation_strategy="epoch",
      output_dir=args_dict["output_dir"],
      overwrite_output_dir=True,
      # The next line is important to ensure the dataset labels are properly passed to the model
      remove_unused_columns=False,
      seed = seed
  )

  trainer = Trainer(
      model = model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_data,
      eval_dataset=val_data
  )


  trainer.train()

  metrics = trainer.evaluate(test_data)
  print("Metrics for seed {}: {}".format(seed, metrics))

  f1_scores.append(metrics["eval_f1"])

  model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name, with_head=True)




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.673208,0.593750,0.372549,0.296875,0.500000,0.713400,89.710000
2,No log,0.667709,0.593750,0.372549,0.296875,0.500000,0.747700,85.594000
3,No log,0.648143,0.593750,0.372549,0.296875,0.500000,0.718500,89.075000
4,No log,0.613120,0.593750,0.372549,0.296875,0.500000,0.722800,88.547000
5,No log,0.519104,0.750000,0.722944,0.758454,0.716599,0.707100,90.517000
6,No log,0.509279,0.734375,0.702326,0.744055,0.697368,0.728900,87.800000
7,No log,0.451364,0.812500,0.787375,0.854167,0.775304,0.708400,90.340000
8,No log,0.480938,0.781250,0.745455,0.834286,0.736842,0.704800,90.810000
9,No log,0.432068,0.812500,0.787375,0.854167,0.775304,0.705400,90.730000
10,No log,0.435657,0.796875,0.766751,0.844218,0.756073,0.721900,88.660000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for seed 1: {'eval_loss': 0.38809365034103394, 'eval_accuracy': 0.8615384615384616, 'eval_f1': 0.8470588235294118, 'eval_precision': 0.9042553191489362, 'eval_recall': 0.8333333333333333, 'eval_runtime': 0.7696, 'eval_samples_per_second': 84.458, 'epoch': 15.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 579047424}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.675588,0.593750,0.372549,0.296875,0.500000,0.712700,89.795000
2,No log,0.663934,0.593750,0.372549,0.296875,0.500000,0.720400,88.835000
3,No log,0.648090,0.593750,0.372549,0.296875,0.500000,0.706200,90.623000
4,No log,0.653748,0.593750,0.372549,0.296875,0.500000,0.707900,90.411000
5,No log,0.517242,0.781250,0.745455,0.834286,0.736842,0.719100,89.005000
6,No log,0.496885,0.796875,0.766751,0.844218,0.756073,0.711500,89.952000
7,No log,0.459481,0.796875,0.772367,0.824155,0.762146,0.723800,88.417000
8,No log,0.414864,0.796875,0.766751,0.844218,0.756073,0.711100,90.003000
9,No log,0.632551,0.765625,0.723423,0.824284,0.717611,0.711200,89.983000
10,No log,0.473287,0.781250,0.745455,0.834286,0.736842,0.704600,90.830000


Metrics for seed 2: {'eval_loss': 0.38007232546806335, 'eval_accuracy': 0.9230769230769231, 'eval_f1': 0.9181153943058704, 'eval_precision': 0.9418604651162791, 'eval_recall': 0.9074074074074074, 'eval_runtime': 0.7735, 'eval_samples_per_second': 84.029, 'epoch': 15.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 579047424}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,0.674596,0.593750,0.372549,0.296875,0.500000,0.723200,88.496000
2,No log,0.663402,0.593750,0.372549,0.296875,0.500000,0.720300,88.851000
3,No log,0.649622,0.593750,0.372549,0.296875,0.500000,0.706900,90.541000
4,No log,0.592817,0.609375,0.413275,0.801587,0.519231,0.711900,89.898000
5,No log,0.515442,0.781250,0.751938,0.812500,0.742915,0.704900,90.794000
6,No log,0.436201,0.781250,0.745455,0.834286,0.736842,0.719200,88.985000
7,No log,0.369268,0.828125,0.811513,0.847368,0.800607,0.711900,89.898000
8,No log,0.428672,0.781250,0.745455,0.834286,0.736842,0.726600,88.084000
9,No log,0.542963,0.796875,0.760300,0.872549,0.750000,0.713500,89.696000
10,No log,0.315441,0.890625,0.885918,0.888718,0.883603,0.718600,89.062000


Metrics for seed 3: {'eval_loss': 0.4328892230987549, 'eval_accuracy': 0.9076923076923077, 'eval_f1': 0.9009146341463414, 'eval_precision': 0.9318181818181819, 'eval_recall': 0.8888888888888888, 'eval_runtime': 0.7755, 'eval_samples_per_second': 83.819, 'epoch': 15.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 780374016}


In [13]:
print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Avg f1 score: 0.8886962839938746, stdev: 0.030267971223321036
